In [1]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df=pd.read_csv(r"D:\Capstone\Datasets\Data_preparation\labels3_data.csv")

In [3]:

df.isnull().sum()

docid     0
text      3
topics    0
dtype: int64

In [4]:
df=df.dropna()
df.isnull().sum()

docid     0
text      0
topics    0
dtype: int64

In [5]:
topic_codes_path = r"D:\Capstone\Datasets\rcv1\rcv1\codes\topics_codes.txt"

code_to_description = {}

with open(topic_codes_path, 'r') as file:
    for line in file:
        # Each line has the format "code description"
        # Splitting by the first space gives you the code and its description
        parts = line.strip().split('\t')
        if len(parts) == 2:
            code, description = parts
            code_to_description[code] = description

# Define a function to replace codes with descriptions
def map_codes_to_descriptions(topic_codes):
    # Split the topic codes, map each one to its description, and join them back together
    return ', '.join(code_to_description.get(code.strip(), code) for code in topic_codes.split(','))

# Apply the function to the 'topics' column
df['topics'] = df['topics'].apply(map_codes_to_descriptions)

# Display the updated DataFrame
print(df)

         docid                                               text  \
0         2289  CompuServe Corp. Tuesday reported a surprising...   
1         2290  If dining at Planet Hollywood made you feel li...   
2         2302  Columbia/HCA Healthcare Corp.'s proposed $299....   
3         2307  World oil prices slipped on Tuesday in a marke...   
4         2312  Gulf War hero Colin Powell lent his prestige a...   
...        ...                                                ...   
362109  810557  Irish independent, low-cost, no-frills airline...   
362110  810558  The latest opinion poll released ahead of the ...   
362111  810574  Australia's oldest newspaper publisher John Fa...   
362112  810575  Concerns about a speculative attack on the Hon...   
362113  810581  The Brown & Williamson Tobacco Co. appears to ...   

                                                   topics  
0       PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/INDU...  
1       STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORP

In [7]:
df.head(10)

,docid,text,topics
0,2289,CompuServe Corp. Tuesday reported a surprising...,"PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/INDU..."
1,2290,If dining at Planet Hollywood made you feel li...,"STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORPORA..."
2,2302,Columbia/HCA Healthcare Corp.'s proposed $299....,"OWNERSHIP_CHANGES, MERGERS/ACQUISITIONS, CORPO..."
3,2307,World oil prices slipped on Tuesday in a marke...,"COMMODITY_MARKETS, ENERGY_MARKETS, MARKETS"
4,2312,Gulf War hero Colin Powell lent his prestige a...,"GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS"
5,2315,The Trocadero entertainment complex in the hea...,"PERFORMANCE, COMMENT/FORECASTS, CORPORATE/INDU..."
6,2316,German utility group Viag AG on Tuesday said i...,"PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/INDU..."
7,2317,Sprint Corp. Tuesday announced plans to offer ...,"STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORPORA..."
8,2320,The New York Stock Exchange said Tuesday that ...,"PRODUCTION/SERVICES, MARKETS/MARKETING, CORPOR..."
9,2322,Silver prices were at their highest levels sin...,"COMMODITY_MARKETS, METALS_TRADING, MARKETS"


In [75]:
document_df=df.copy()

In [25]:
# Assuming df['topics'] contains strings with topics separated by commas
document_df['topics'] = document_df['topics'].str.split(', ')
document_df.shape

(362111, 3)

In [26]:
document_df['topics']

0         [PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND...
1         [STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORPOR...
2         [OWNERSHIP_CHANGES, MERGERS/ACQUISITIONS, CORP...
3              [COMMODITY_MARKETS, ENERGY_MARKETS, MARKETS]
4         [GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]
                                ...                        
362109    [MARKETS/MARKETING, CONTRACTS/ORDERS, CORPORAT...
362110    [GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]
362111    [PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND...
362112    [COMMENT/FORECASTS, DISASTERS_AND_ACCIDENTS, E...
362113    [LEGAL/JUDICIAL, RESEARCH/DEVELOPMENT, CRIME,_...
Name: topics, Length: 362111, dtype: object

In [27]:
from collections import Counter
topic_counts = Counter([topic for sublist in document_df['topics'] for topic in sublist])

# Remove topics with counts less than 200
topics_to_remove = {topic for topic, count in topic_counts.items() if count < 500}
topics_to_remove

{'ANNUAL_RESULTS',
 'BOND_MARKETS',
 'CAPACITY_UTILIZATION',
 'CONSUMER_CREDIT',
 'DEFENCE_CONTRACTS',
 'DOMESTIC_MARKETS',
 'EC_AGRICULTURE_POLICY',
 'EC_COMPETITION/SUBSIDY',
 'EC_CORPORATE_POLICY',
 'EC_ENVIRONMENT_ISSUES',
 'EC_EXTERNAL_RELATIONS',
 'EC_GENERAL',
 'EC_INSTITUTIONS',
 'EC_INTERNAL_MARKET',
 'EC_MONETARY/ECONOMIC',
 'EMPLOYMENT/LABOUR',
 'FASHION',
 'HOUSING_STARTS',
 'INSOLVENCY/LIQUIDITY',
 'INVENTORIES',
 'LABOUR',
 'LABOUR_ISSUES',
 'LEADING_INDICATORS',
 'LEGAL/JUDICIAL',
 'MARKET_SHARE',
 'OBITUARIES',
 'PERSONAL_INCOME',
 'SHARE_LISTINGS',
 'SPORTS',
 'TRAVEL_AND_TOURISM',
 'UNEMPLOYMENT',
 'WELFARE,_SOCIAL_SERVICES'}

In [28]:
topics = [topic for topic, _ in topic_counts.items() if topic not in topics_to_remove]
len(set(topics))

70

In [29]:
topics

['PERFORMANCE',
 'ACCOUNTS/EARNINGS',
 'CORPORATE/INDUSTRIAL',
 'STRATEGY/PLANS',
 'NEW_PRODUCTS/SERVICES',
 'OWNERSHIP_CHANGES',
 'MERGERS/ACQUISITIONS',
 'COMMODITY_MARKETS',
 'ENERGY_MARKETS',
 'MARKETS',
 'GOVERNMENT/SOCIAL',
 'DOMESTIC_POLITICS',
 'ELECTIONS',
 'COMMENT/FORECASTS',
 'PRODUCTION/SERVICES',
 'MARKETS/MARKETING',
 'METALS_TRADING',
 'INFLATION/PRICES',
 'CONSUMER_PRICES',
 'ECONOMICS',
 'TRADE/RESERVES',
 'MERCHANDISE_TRADE',
 'CAPACITY/FACILITIES',
 'REGULATION/POLICY',
 'WAR,_CIVIL_WAR',
 'CRIME,_LAW_ENFORCEMENT',
 'MONEY_MARKETS',
 'FOREX_MARKETS',
 'GOVERNMENT_FINANCE',
 'EXPENDITURE/REVENUE',
 'ECONOMIC_PERFORMANCE',
 'MONETARY/ECONOMIC',
 'CONTRACTS/ORDERS',
 'FUNDING/CAPITAL',
 'SHARE_CAPITAL',
 'BONDS/DEBT_ISSUES',
 'INTERBANK_MARKETS',
 'MONEY_SUPPLY',
 'MONOPOLIES/COMPETITION',
 'MANAGEMENT',
 'MANAGEMENT_MOVES',
 'ASSET_TRANSFERS',
 'ADVERTISING/PROMOTION',
 'LOANS/CREDITS',
 'CONSUMER_FINANCE',
 'RETAIL_SALES',
 'SOFT_COMMODITIES',
 'HUMAN_INTEREST',
 'IN

In [21]:
document_df['topics'] = document_df['topics'].apply(lambda topics: [topic for topic in topics if topic not in topics_to_remove])
print(document_df.shape)
document_df.head(10)

(362111, 3)


,docid,text,topics
0,2289,CompuServe Corp. Tuesday reported a surprising...,"[PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND..."
1,2290,If dining at Planet Hollywood made you feel li...,"[STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORPOR..."
2,2302,Columbia/HCA Healthcare Corp.'s proposed $299....,"[OWNERSHIP_CHANGES, MERGERS/ACQUISITIONS, CORP..."
3,2307,World oil prices slipped on Tuesday in a marke...,"[COMMODITY_MARKETS, ENERGY_MARKETS, MARKETS]"
4,2312,Gulf War hero Colin Powell lent his prestige a...,"[GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]"
5,2315,The Trocadero entertainment complex in the hea...,"[PERFORMANCE, COMMENT/FORECASTS, CORPORATE/IND..."
6,2316,German utility group Viag AG on Tuesday said i...,"[PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND..."
7,2317,Sprint Corp. Tuesday announced plans to offer ...,"[STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORPOR..."
8,2320,The New York Stock Exchange said Tuesday that ...,"[PRODUCTION/SERVICES, MARKETS/MARKETING, CORPO..."
9,2322,Silver prices were at their highest levels sin...,"[COMMODITY_MARKETS, METALS_TRADING, MARKETS]"


In [30]:
# Count the number of topics before removal
original_topic_count = document_df['topics'].apply(len).sum()

# Apply removal operation
document_df['topics'] = document_df['topics'].apply(lambda topics: [topic for topic in topics if topic not in topics_to_remove])

# Count the number of topics after removal
new_topic_count = document_df['topics'].apply(len).sum()

# Calculate the count of removed topics
removed_topic_count = original_topic_count - new_topic_count

print("Number of topics removed:", removed_topic_count)


Number of topics removed: 5422


In [31]:
print(original_topic_count)
print(removed_topic_count)

1086333
5422


In [32]:
document_df['count_topics'] = document_df['topics'].apply(lambda x: len(x))
document_df['count_topics'].value_counts()

count_topics
3    357359
2      4148
1       538
0        66
Name: count, dtype: int64

In [47]:
train_set = document_df[(document_df['count_topics'] != 0)| (document_df['count_topics'] != 1)]
train_set.shape

(362111, 4)

In [23]:
adjusted_stopwords = [
    'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 
    'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'could','couldn', "couldn't", 
    'could not','cannot', 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 
    'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 
    'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', 
    "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', 
    "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 
    'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', 
    "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 
    'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 
    'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 
    'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 
    'yourself', 'yourselves', 'did not', 'does not', 'do not', 'had not', 'has not', 'have not', 'he is', 'she is', 'it is', 'we are', 
    'they are', 'you are', 'I am', 'should not', 'would not', 'shall not', 'will not','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 
    'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

adjusted_stopwords_set = set(adjusted_stopwords)
    
titles = ['Mr\.', 'Mrs\.', 'Miss', 'Ms\.', 'Mx\.', 'Dr\.', 'Prof\.', 'Sir', 'Lady', 'Lord', 'Rev\.', 'Fr\.', 'Rabbi', 'Imam', 'Capt\.', 'Maj\.', 'Lt\.', 'Col\.', 'Gen\.', 'Adm\.', 'Esq\.']
    


In [24]:
def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

def remove_punctuation(text):
    text = re.sub(r"(\w+)'s", r"\1", text)
    text = re.sub(r'\d+', '', text).lower()
    text = re.sub(r'(\b\w+)-(\w+\b)', r'\1 \2', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text
    
def remove_titles(text):
    title_pattern = '|'.join(titles)
    return re.sub(title_pattern, '', text)

def tokenize(text):
    split = re.split("\W+", text) 
    return split

def remove_stopwords(text_tokens):
    filtered_text = [word for word in text_tokens if word.lower() not in adjusted_stopwords_set]
    return filtered_text


def lemmatize_text(text_tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(token, pos='v') if token is not None else '' for token in text_tokens]
    return lemmatized




In [26]:
import contractions

def preprocess_text(df):
    
    df['text_expanded'] = df['text'].apply(lambda x: expand_contractions(x))
    
    
    df['text_wo_punct'] = df['text_expanded'].apply(lambda x: remove_punctuation(x))
    
    
    df['remove_titles'] = df['text_wo_punct'].apply(lambda x: remove_titles(x.lower()))
    
    
    df['text_tokenized'] = df['remove_titles'].apply(lambda x: tokenize(x.lower()))
    
    
    df['text_wo_stopwords'] = df['text_tokenized'].apply(lambda x: remove_stopwords(x))
    
    
    df['text_lemmatized'] = df['text_wo_stopwords'].apply(lambda x: lemmatize_text(x))
    
    return df

train_set_df = preprocess_text(train_set)


In [29]:
train_set_df

,docid,text,topics,count_topics,text_expanded,text_wo_punct,remove_titles,text_tokenized,text_wo_stopwords,text_lemmatized
0,2289,CompuServe Corp. Tuesday reported a surprising...,"[PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND...",3,CompuServe Corp. Tuesday reported a surprising...,compuserve corp tuesday reported a surprisingl...,compuserve corp tuesday reported a surprisingl...,"[compuserve, corp, tuesday, reported, a, surpr...","[compuserve, corp, tuesday, reported, surprisi...","[compuserve, corp, tuesday, report, surprising..."
1,2290,If dining at Planet Hollywood made you feel li...,"[STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORPOR...",3,If dining at Planet Hollywood made you feel li...,if dining at planet hollywood made you feel li...,if dining at planet hollywood made you feel li...,"[if, dining, at, planet, hollywood, made, you,...","[dining, planet, hollywood, made, feel, like, ...","[din, planet, hollywood, make, feel, like, mov..."
2,2302,Columbia/HCA Healthcare Corp.'s proposed $299....,"[OWNERSHIP_CHANGES, MERGERS/ACQUISITIONS, CORP...",3,Columbia/HCA Healthcare Corp.'s proposed $299....,columbiahca healthcare corps proposed million...,columbiahca healthcare corps proposed million...,"[columbiahca, healthcare, corps, proposed, mil...","[columbiahca, healthcare, corps, proposed, mil...","[columbiahca, healthcare, corps, propose, mill..."
3,2307,World oil prices slipped on Tuesday in a marke...,"[COMMODITY_MARKETS, ENERGY_MARKETS, MARKETS]",3,World oil prices slipped on Tuesday in a marke...,world oil prices slipped on tuesday in a marke...,world oil prices slipped on tuesday in a marke...,"[world, oil, prices, slipped, on, tuesday, in,...","[world, oil, prices, slipped, tuesday, market,...","[world, oil, price, slip, tuesday, market, ref..."
4,2312,Gulf War hero Colin Powell lent his prestige a...,"[GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]",3,Gulf War hero Colin Powell lent his prestige a...,gulf war hero colin powell lent his prestige a...,gulf war hero colin powell lent his prestige a...,"[gulf, war, hero, colin, powell, lent, his, pr...","[gulf, war, hero, colin, powell, lent, prestig...","[gulf, war, hero, colin, powell, lend, prestig..."
...,...,...,...,...,...,...,...,...,...,...
362109,810557,"Irish independent, low-cost, no-frills airline...","[MARKETS/MARKETING, CONTRACTS/ORDERS, CORPORAT...",3,"Irish independent, low-cost, no-frills airline...",irish independent low cost no frills airline r...,irish independent low cost no frills airline r...,"[irish, independent, low, cost, no, frills, ai...","[irish, independent, low, cost, frills, airlin...","[irish, independent, low, cost, frills, airlin..."
362110,810558,The latest opinion poll released ahead of the ...,"[GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]",3,The latest opinion poll released ahead of the ...,the latest opinion poll released ahead of the ...,the latest opinion poll released ahead of the ...,"[the, latest, opinion, poll, released, ahead, ...","[latest, opinion, poll, released, ahead, septe...","[latest, opinion, poll, release, ahead, septem..."
362111,810574,Australia's oldest newspaper publisher John Fa...,"[PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND...",3,Australia's oldest newspaper publisher John Fa...,australia oldest newspaper publisher john fair...,australia oldest newspaper publisher john fair...,"[australia, oldest, newspaper, publisher, john...","[australia, oldest, newspaper, publisher, john...","[australia, oldest, newspaper, publisher, john..."
362112,810575,Concerns about a speculative attack on the Hon...,"[COMMENT/FORECASTS, DISASTERS_AND_ACCIDENTS, E...",3,Concerns about a speculative attack on the Hon...,concerns about a speculative attack on the hon...,concerns about a speculative attack on the hon...,"[concerns, about, a, speculative, attack, on, ...","[concerns, speculative, attack, hong, kong, do...","[concern, speculative, attack, hong, kong, dol..."


In [30]:
df_cleaned = train_set_df.copy()

In [31]:
df_cleaned = df_cleaned.reset_index(drop=True)
.shape

(362111, 10)

In [36]:
df_cleaned['text_lemmatized']

0         [compuserve, corp, tuesday, report, surprising...
1         [din, planet, hollywood, make, feel, like, mov...
2         [columbiahca, healthcare, corps, propose, mill...
3         [world, oil, price, slip, tuesday, market, ref...
4         [gulf, war, hero, colin, powell, lend, prestig...
                                ...                        
362106    [irish, independent, low, cost, frills, airlin...
362107    [latest, opinion, poll, release, ahead, septem...
362108    [australia, oldest, newspaper, publisher, john...
362109    [concern, speculative, attack, hong, kong, dol...
362110    [brown, williamson, tobacco, co, appear, devel...
Name: text_lemmatized, Length: 362111, dtype: object

In [37]:
import ast


# Now you can proceed with concatenating the lists into sentences
df_cleaned['processed_text'] = df_cleaned['text_lemmatized'].apply(lambda x: ' '.join(x))


In [32]:
from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate the binarizer
mlb = MultiLabelBinarizer()

# Fit and transform the 'Topics' column of the DataFrame
binary_labels = mlb.fit_transform(df_cleaned['topics'])

# Create a new DataFrame from the binary labels with appropriate column names
mlb_df = pd.DataFrame(binary_labels, columns=mlb.classes_, index=train_set.index)
mlb_df.head(3)

,ACCOUNTS/EARNINGS,ADVERTISING/PROMOTION,"ARTS,_CULTURE,_ENTERTAINMENT",ASSET_TRANSFERS,BALANCE_OF_PAYMENTS,"BIOGRAPHIES,_PERSONALITIES,_PEOPLE",BONDS/DEBT_ISSUES,CAPACITY/FACILITIES,COMMENT/FORECASTS,COMMODITY_MARKETS,...,RESERVES,RETAIL_SALES,SCIENCE_AND_TECHNOLOGY,SHARE_CAPITAL,SOFT_COMMODITIES,STRATEGY/PLANS,TRADE/RESERVES,"WAR,_CIVIL_WAR",WEATHER,WHOLESALE_PRICES
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
X_documents = df_cleaned['processed_text'].to_list()
y = mlb_df.values

len(X_documents), y.shape

(362111, (362111, 70))

In [39]:
df_cleaned['processed_text']

0         compuserve corp tuesday report surprisingly la...
1         din planet hollywood make feel like movie star...
2         columbiahca healthcare corps propose million a...
3         world oil price slip tuesday market refiners s...
4         gulf war hero colin powell lend prestige popul...
                                ...                        
362106    irish independent low cost frills airline ryan...
362107    latest opinion poll release ahead september no...
362108    australia oldest newspaper publisher john fair...
362109    concern speculative attack hong kong dollar ro...
362110    brown williamson tobacco co appear develop det...
Name: processed_text, Length: 362111, dtype: object

In [40]:
X_documents

['compuserve corp tuesday report surprisingly large million fiscal first quarter loss blame decline number subscribers online service spend new family orient service improvements compuserve predict second quarter loss say earn would improve second half fiscal year take step revitalise compuserve say chief executive bob massey company say would cut job percent work force part cost cut programme expect save million annualized basis columbus ohio base company say part cost cut would sell spry inc unit whose web browser trail far behind similar products make netscape communications corp microsoft corp loss million cents share company fiscal first quarter end july compare last year quarterly profit million cents share result include pretax charge million cents per share compuserve last month say expect post loss quarter actual loss greater wall street expectations company report revenues million quarter vs million previous year quarter massey expand last month comment say number people canc

In [41]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [42]:
df_cleaned

,docid,text,topics,count_topics,text_expanded,text_wo_punct,remove_titles,text_tokenized,text_wo_stopwords,text_lemmatized,processed_text
0,2289,CompuServe Corp. Tuesday reported a surprising...,"[PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND...",3,CompuServe Corp. Tuesday reported a surprising...,compuserve corp tuesday reported a surprisingl...,compuserve corp tuesday reported a surprisingl...,"[compuserve, corp, tuesday, reported, a, surpr...","[compuserve, corp, tuesday, reported, surprisi...","[compuserve, corp, tuesday, report, surprising...",compuserve corp tuesday report surprisingly la...
1,2290,If dining at Planet Hollywood made you feel li...,"[STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORPOR...",3,If dining at Planet Hollywood made you feel li...,if dining at planet hollywood made you feel li...,if dining at planet hollywood made you feel li...,"[if, dining, at, planet, hollywood, made, you,...","[dining, planet, hollywood, made, feel, like, ...","[din, planet, hollywood, make, feel, like, mov...",din planet hollywood make feel like movie star...
2,2302,Columbia/HCA Healthcare Corp.'s proposed $299....,"[OWNERSHIP_CHANGES, MERGERS/ACQUISITIONS, CORP...",3,Columbia/HCA Healthcare Corp.'s proposed $299....,columbiahca healthcare corps proposed million...,columbiahca healthcare corps proposed million...,"[columbiahca, healthcare, corps, proposed, mil...","[columbiahca, healthcare, corps, proposed, mil...","[columbiahca, healthcare, corps, propose, mill...",columbiahca healthcare corps propose million a...
3,2307,World oil prices slipped on Tuesday in a marke...,"[COMMODITY_MARKETS, ENERGY_MARKETS, MARKETS]",3,World oil prices slipped on Tuesday in a marke...,world oil prices slipped on tuesday in a marke...,world oil prices slipped on tuesday in a marke...,"[world, oil, prices, slipped, on, tuesday, in,...","[world, oil, prices, slipped, tuesday, market,...","[world, oil, price, slip, tuesday, market, ref...",world oil price slip tuesday market refiners s...
4,2312,Gulf War hero Colin Powell lent his prestige a...,"[GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]",3,Gulf War hero Colin Powell lent his prestige a...,gulf war hero colin powell lent his prestige a...,gulf war hero colin powell lent his prestige a...,"[gulf, war, hero, colin, powell, lent, his, pr...","[gulf, war, hero, colin, powell, lent, prestig...","[gulf, war, hero, colin, powell, lend, prestig...",gulf war hero colin powell lend prestige popul...
...,...,...,...,...,...,...,...,...,...,...,...
362106,810557,"Irish independent, low-cost, no-frills airline...","[MARKETS/MARKETING, CONTRACTS/ORDERS, CORPORAT...",3,"Irish independent, low-cost, no-frills airline...",irish independent low cost no frills airline r...,irish independent low cost no frills airline r...,"[irish, independent, low, cost, no, frills, ai...","[irish, independent, low, cost, frills, airlin...","[irish, independent, low, cost, frills, airlin...",irish independent low cost frills airline ryan...
362107,810558,The latest opinion poll released ahead of the ...,"[GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]",3,The latest opinion poll released ahead of the ...,the latest opinion poll released ahead of the ...,the latest opinion poll released ahead of the ...,"[the, latest, opinion, poll, released, ahead, ...","[latest, opinion, poll, released, ahead, septe...","[latest, opinion, poll, release, ahead, septem...",latest opinion poll release ahead september no...
362108,810574,Australia's oldest newspaper publisher John Fa...,"[PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND...",3,Australia's oldest newspaper publisher John Fa...,australia oldest newspaper publisher john fair...,australia oldest newspaper publisher john fair...,"[australia, oldest, newspaper, publisher, john...","[australia, oldest, newspaper, publisher, john...","[australia, oldest, newspaper, publisher, john...",australia oldest newspaper publisher john fair...
362109,810575,Concerns about a speculative attack on the Hon

In [43]:
df_cleaned.to_csv(r'D:\Capstone\Datasets\Data_preparation\preprocessed_labels3_data.csv', index=False)

In [44]:

mlb_df.to_csv(r'D:\Capstone\Datasets\Data_preparation\mlb_df.csv', index=False)

In [38]:
train_set['topics'].unique

<bound method Series.unique of 0         [PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND...
1         [STRATEGY/PLANS, NEW_PRODUCTS/SERVICES, CORPOR...
2         [OWNERSHIP_CHANGES, MERGERS/ACQUISITIONS, CORP...
3              [COMMODITY_MARKETS, ENERGY_MARKETS, MARKETS]
4         [GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]
                                ...                        
362109    [MARKETS/MARKETING, CONTRACTS/ORDERS, CORPORAT...
362110    [GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]
362111    [PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/IND...
362112    [COMMENT/FORECASTS, DISASTERS_AND_ACCIDENTS, E...
362113       [RESEARCH/DEVELOPMENT, CRIME,_LAW_ENFORCEMENT]
Name: topics, Length: 362111, dtype: object>

In [48]:
train_set['topics'].value_counts().head(20)

topics
[PERFORMANCE, COMMENT/FORECASTS, CORPORATE/INDUSTRIAL]             53939
[PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/INDUSTRIAL]             50740
[COMMODITY_MARKETS, SOFT_COMMODITIES, MARKETS]                     36090
[OWNERSHIP_CHANGES, MERGERS/ACQUISITIONS, CORPORATE/INDUSTRIAL]    23106
[MONEY_MARKETS, INTERBANK_MARKETS, MARKETS]                        20439
[GOVERNMENT_FINANCE, GOVERNMENT_BORROWING, ECONOMICS]              17671
[COMMODITY_MARKETS, ENERGY_MARKETS, MARKETS]                       17271
[MONEY_MARKETS, FOREX_MARKETS, MARKETS]                            15811
[FUNDING/CAPITAL, BONDS/DEBT_ISSUES, CORPORATE/INDUSTRIAL]          9489
[COMMODITY_MARKETS, METALS_TRADING, MARKETS]                        7750
[MANAGEMENT, MANAGEMENT_MOVES, CORPORATE/INDUSTRIAL]                7698
[GOVERNMENT/SOCIAL, DOMESTIC_POLITICS, ELECTIONS]                   7080
[FUNDING/CAPITAL, SHARE_CAPITAL, CORPORATE/INDUSTRIAL]              6852
[GOVERNMENT/SOCIAL, INTERNATIONAL_RELATIONS,

In [49]:
train_set['topics'].explode().nunique()

70

In [50]:
train_set['topics'].value_counts().head(60) #52 pairs above 500

topics
[PERFORMANCE, COMMENT/FORECASTS, CORPORATE/INDUSTRIAL]                             53939
[PERFORMANCE, ACCOUNTS/EARNINGS, CORPORATE/INDUSTRIAL]                             50740
[COMMODITY_MARKETS, SOFT_COMMODITIES, MARKETS]                                     36090
[OWNERSHIP_CHANGES, MERGERS/ACQUISITIONS, CORPORATE/INDUSTRIAL]                    23106
[MONEY_MARKETS, INTERBANK_MARKETS, MARKETS]                                        20439
[GOVERNMENT_FINANCE, GOVERNMENT_BORROWING, ECONOMICS]                              17671
[COMMODITY_MARKETS, ENERGY_MARKETS, MARKETS]                                       17271
[MONEY_MARKETS, FOREX_MARKETS, MARKETS]                                            15811
[FUNDING/CAPITAL, BONDS/DEBT_ISSUES, CORPORATE/INDUSTRIAL]                          9489
[COMMODITY_MARKETS, METALS_TRADING, MARKETS]                                        7750
[MANAGEMENT, MANAGEMENT_MOVES, CORPORATE/INDUSTRIAL]                                7698
[GOVERNMENT/SO

In [73]:

top_52_topics = df['topics'].value_counts().head(52)

# Create an empty DataFrame to store sampled data
sampled_data = pd.DataFrame(columns=df.columns)

for topic, count in top_52_topics.items():
    # Filter rows for the current topic
    topic_data = df[df['topics'] == topic]
    
    # If there are more than 500 samples, randomly select 500
    if count > 500:
        topic_data = topic_data.sample(n=500, random_state=42)
    
    # Append selected samples to the sampled_data DataFrame
    sampled_data = pd.concat([sampled_data, topic_data])

# Reset index of the sampled data
sampled_data.reset_index(drop=True, inplace=True)

# Print the shape of the sampled data
print("Shape of sampled data:", sampled_data.shape)


Shape of sampled data: (25957, 3)


In [74]:
#sampled_data['topics']=sampled_data['topics'].str.split(', ')
sampled_data

,docid,text,topics
0,116198,Measurement Specialties Inc said on Tuesday it...,"PERFORMANCE, COMMENT/FORECASTS, CORPORATE/INDU..."
1,613541,Polish edible fat producer Kruszwica SA revise...,"PERFORMANCE, COMMENT/FORECASTS, CORPORATE/INDU..."
2,591893,"Year to March 31, 1998 \t\t (in billions of ...","PERFORMANCE, COMMENT/FORECASTS, CORPORATE/INDU..."
3,128120,"Six months to September 30, 1996 \t\t (in bi...","PERFORMANCE, COMMENT/FORECASTS, CORPORATE/INDU..."
4,192887,"Year to March 31, 1997 \t\t (in billions of ...","PERFORMANCE, COMMENT/FORECASTS, CORPORATE/INDU..."
...,...,...,...
25952,796979,An Indian army soldier ran amok with a gun and...,"GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DEF..."
25953,800402,An English tourist on Thursday identified in c...,"GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DEF..."
25954,805555,A Mongolian court has ruled that former espion...,"GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DEF..."
25955,806688,An Englishwoman broke down in a Cyprus court o...,"GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DEF..."


In [69]:
sampled_data = sampled_data.reset_index(drop=True)
sampled_data.shape

(25957, 3)

In [72]:
sampled_data['topics']

0        [PERFORMANCE, COMMENT/FORECASTS, CORPORATE/IND...
1        [PERFORMANCE, COMMENT/FORECASTS, CORPORATE/IND...
2        [PERFORMANCE, COMMENT/FORECASTS, CORPORATE/IND...
3        [PERFORMANCE, COMMENT/FORECASTS, CORPORATE/IND...
4        [PERFORMANCE, COMMENT/FORECASTS, CORPORATE/IND...
                               ...                        
25952    [GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DE...
25953    [GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DE...
25954    [GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DE...
25955    [GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DE...
25956    [GOVERNMENT/SOCIAL, CRIME,_LAW_ENFORCEMENT, DE...
Name: topics, Length: 25957, dtype: object

In [70]:
from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate the binarizer
mlb = MultiLabelBinarizer()

# Fit and transform the 'topics' column of the DataFrame
binary_labels = mlb.fit_transform(sampled_data['topics'])

# Create a new DataFrame from the binary labels with appropriate column names
mlb_df = pd.DataFrame(binary_labels, columns=mlb.classes_, index=sampled_data.index)
mlb_df.head(3)


,ACCOUNTS/EARNINGS,ASSET_TRANSFERS,BALANCE_OF_PAYMENTS,"BIOGRAPHIES,_PERSONALITIES,_PEOPLE",BONDS/DEBT_ISSUES,CAPACITY/FACILITIES,COMMENT/FORECASTS,COMMODITY_MARKETS,CONSUMER_FINANCE,CONSUMER_PRICES,...,REGULATION/POLICY,RESERVES,RETAIL_SALES,SHARE_CAPITAL,SOFT_COMMODITIES,STRATEGY/PLANS,TRADE/RESERVES,"WAR,_CIVIL_WAR",WEATHER,WHOLESALE_PRICES
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
print(mlb.classes_)


[' ' ',' '/' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'K' 'L' 'M' 'N' 'O' 'P'
 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' '_']


In [67]:
# Create a dictionary to map encoded classes to original column names
class_to_column = {}
for encoded_class, original_column in zip(mlb.classes_, sampled_data.columns):
    class_to_column[encoded_class] = original_column

# Print the dictionary
print(class_to_column)


{' ': 'docid', ',': 'text', '/': 'topics'}
